## Temat 1. Własna sieć neuronowa
Celem projektu jest samodzielne zaimplementowanie prostej sieci neuronowej i przetestowanie jej na wybranym przez siebie przykładzie.

Podstawowym założeniem projektu jest własnoręczne zaimplementowanie automatycznego różniczkowania w celu wyznaczania pochodnych warstw potrzebnych do wstecznej   propagacji błędów. Dopuszczalne jest wykorzystanie różniczkowanie w przód (ang. Forward Accumulation), różniczkowanie w tył (ang. Reverse Accumulation) oraz różniczkowanie oparte o generację kodu (ang. source-to-source differentiation).

Do implementacji optymalizatorów dokonujących właściwego uczenia sieci można wykorzystać kod zamieszczony w książce "Algorithms for Optimization" [1].

klasyfikacja cyfr (Digits MNIST dataset)  
Bibliografia:  
[1] Mykel J. Kochenderfer, Tim A. Wheeler, 2019, Algorithms for Optimization, MIT Press.  
[2] Martin T. Hagan et. al, Neural Network Design, pp. 915-918, url: https://hagan.okstate.edu/NNDesign.pdf  
[3] 3blue1brown: czym są sieci neuronowe? https://www.youtube.com/watch?v=aircAruvnKk  
  


In [37]:
using Pkg
Pkg.activate("./project")

  Activating project at `e:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project`


In [38]:
Pkg.add("MLDatasets")
Pkg.add("IJulia")
Pkg.add("StableRNGs")
Pkg.add("DataFrames")
Pkg.add("PyPlot")
ENV["PYTHON"]=""

   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


   Resolving package versions...


...


   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


   Resolving package versions...


  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Project.toml`
  No Changes to `E:\Documents\1. Studia\MAGISTERSKIE\Algorytmy w inż danych\Custom-Julia-NN\project\Manifest.toml`


""

# Load Dataset

In [39]:
using MLDatasets
using StableRNGs
train_x, train_y = MNIST.traindata(Float32);
test_x, test_y = MNIST.testdata(Float32);
Y_labels = [0,1,2,3,4,5,6,7,8,9]
train_size = size(train_x)
test_size = size(test_x)
train_x = reshape(train_x, (train_size[1],train_size[2],1,train_size[3]))
test_x = reshape(test_x, (test_size[1],test_size[2],1,test_size[3]))
println("Train X shape: ",size(train_x));
println("Test X shape: ",size((test_x)));
test_test_y = test_y
test_y= Y_labels  .== permutedims(test_y)
train_y= Y_labels .== permutedims(train_y)
println("Train Y shape: ",size(train_y));
println("Test Y shape: ",size((test_y)));

Train X shape: (28, 28, 1, 60000)
Test X shape: (28, 28, 1, 10000)
Train Y shape: (10, 60000)
Test Y shape: (10, 10000)


###  Automatic differentation- foward accumulation

In [40]:
# Dual number
struct Dual{T <:Number} <:Number
     v::T
    dv::T
end


### Overloading functions and operators

In [41]:
import Base: +, -, *, /
-(x::Dual)          = Dual(-x.v,       -x.dv)
+(x::Dual, y::Dual) = Dual( x.v + y.v,  x.dv + y.dv)
-(x::Dual, y::Dual) = Dual( x.v - y.v,  x.dv - y.dv)
*(x::Dual, y::Dual) = Dual( x.v * y.v,  x.dv * y.v + x.v * y.dv)
/(x::Dual, y::Dual) = Dual( x.v / y.v, (x.dv * y.v - x.v * y.dv)/y.v^2)

import Base: abs, sin, cos, tan, exp, sqrt, isless
abs(x::Dual)  = Dual(abs(x.v),sign(x.v)*x.dv)
sin(x::Dual)  = Dual(sin(x.v), cos(x.v)*x.dv)
cos(x::Dual)  = Dual(cos(x.v),-sin(x.v)*x.dv)
tan(x::Dual)  = Dual(tan(x.v), one(x.v)*x.dv + tan(x.v)^2*x.dv)
exp(x::Dual)  = Dual(exp(x.v), exp(x.v)*x.dv)
sqrt(x::Dual) = Dual(sqrt(x.v),.5/sqrt(x.v) * x.dv)
isless(x::Dual, y::Dual) = x.v < y.v;


In [33]:
import Base: convert, promote_rule

convert(::Type{Dual{T}}, x::Dual) where T = Dual(convert(T, x.v), convert(T, x.dv))
@show Dual{Float64}[Dual(1,2), Dual(3,0)];
convert(::Type{Dual{T}}, x::Number) where T = Dual(convert(T, x), zero(T))
@show Dual{Float64}[1, 2, 3];
promote_rule(::Type{Dual{T}}, ::Type{R}) where {T,R} = Dual{promote_type(T,R)}
@show Dual(1,2) * 3;

import Base: show
show(io::IO, x::Dual) = print(io, "(", x.v, ") + [", x.dv, "ϵ]");
value(x::Dual) = x.v;
partials(x::Dual) = x.dv;

Dual{Float64}[Dual(1, 2), Dual(3, 0)] = Dual{Float64}[(1.0) + [2.0ϵ], (3.0) + [0.0ϵ]]
Dual{Float64}[1, 2, 3] = Dual{Float64}[(1.0) + [0.0ϵ], (2.0) + [0.0ϵ], (3.0) + [0.0ϵ]]
Dual(1, 2) * 3 = (3) + [6ϵ]


In [34]:
D = derivative(f, x) = partials(f(Dual(x, one(x))))

derivative (generic function with 1 method)

In [35]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        column = partials.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

jacobian (generic function with 1 method)

In [36]:
H = function hessian(f, args::Vector)
    ∇f(x::Vector) = J(f, x)
    J(∇f, args)
end

hessian (generic function with 1 method)

In [14]:
import LinearAlgebra: diagm
diagonal(m) = diagm(0 => vec(m))

### Activation functions

In [22]:
softmax(x)  =  exp.(x) ./ sum(exp.(x));
dsoftmax(x) = (softmax(x) |> diagonal) .- softmax(x) * (softmax(x) |> transpose);

ReLU(x) = max(zero(x), x)
σ(x) = one(x) / (one(x) + exp(-x))
tanh(x) = 2.0 / (one(x) + exp(-2.0x)) - one(x)

tanh (generic function with 1 method)

### Cost functions

In [128]:
mean_squared_loss(y::Vector, ŷ::Vector) = sum(0.5(y - ŷ).^2)
binary_cross_entropy(y::Vector, ŷ::Vector) = let 
    epsilon = eps(1.0)
    ## Avoding 0 , 1 in log argument
    ŷ = [max(i, epsilon) for i in ŷ]
    ŷ = [min(i, 1-epsilon) for i in ŷ]
    return -sum(y .* log.(ŷ) + (1 .- y) .* log.(1 .- ŷ)) / length(y)
end

binary_cross_entropy (generic function with 1 method)

In [143]:
@show binary_cross_entropy([1,1,0.01],[1,1,0])
@show mean_squared_loss([1,1,1],[0,0,0])

binary_cross_entropy([1, 1, 0.01], [1, 1, 0]) = 0.1201455112970574
mean_squared_loss([1, 1, 1], [0, 0, 0]) = 1.5


1.5

### Net layers

In [145]:
fullyconnected(w::Vector, n::Number, m::Number, v::Vector, activation::Function) = activation.(reshape(w, n, m) * v)

fullyconnected (generic function with 1 method)

### Network initialization

In [149]:
Wh  = randn(10,2)
Wo  = randn(1,10)
dWh = similar(Wh)
dWo = similar(Wo)
x = [1.98;4.434]
y = [0.064]
E = Float64[]

function net(x, wh, wo, y)
    x̂ = fullyconnected(wh, 10, 2, x, σ)
    ŷ = fullyconnected(wo, 1, 10, x̂, u->u)
    E = mean_squared_loss(y, ŷ)
end
Ei = net(x, Wh[:], Wo[:], y)

0.7820978145137796

In [158]:
dnet_Wh(x, wh, wo, y) = J(w -> net(x, w, wo, y), wh);
dWh[:] = dnet_Wh(x, Wh[:], Wo[:], y);

dnet_Wo(x, wh, wo, y) = J(w -> net(x, wh, w, y), wo);
dWo[:] = dnet_Wo(x, Wh[:], Wo[:], y);